In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# Filtrado del conjunto de entrenamiento

In [47]:
dataDir='/home/sergio/Documentos/COCO_Dataset'
dataType='train2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
print(annFile)

/home/sergio/Documentos/COCO_Dataset/annotations/instances_train2017.json


In [48]:
from pycocotools.coco import COCO
import numpy as np
#import skimage.io as io
import matplotlib.pyplot as plt
#import pylab

# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=8.69s)
creating index...
index created!


In [4]:
catIds = coco.getCatIds(catNms=['person']);
imgIds = coco.getImgIds(catIds=catIds );

In [5]:
aux = coco.loadImgs(imgIds)
real_names =[]
for i in aux:
    real_names.append(i["file_name"])
print(len(real_names))
print(real_names[:12])

64115
['000000262145.jpg', '000000262146.jpg', '000000524291.jpg', '000000262148.jpg', '000000393223.jpg', '000000393224.jpg', '000000524297.jpg', '000000393227.jpg', '000000131084.jpg', '000000393230.jpg', '000000262161.jpg', '000000131089.jpg']


In [6]:
import os
os.chdir("/home/sergio/Documentos/COCO_Dataset")
os.mkdir("pedestrian_dataset_train")

FileExistsError: [Errno 17] File exists: 'pedestrian_dataset_train'

In [10]:
from shutil import copyfile

def get_class_images(SOURCE, NEW_LOC,CLASS_NAMES):

    names_list =os.listdir(SOURCE)

    for i in names_list:
        if i in CLASS_NAMES:
                copyfile(SOURCE + "/" + i, NEW_LOC + "/" + i)

#dict_pedestrian = get_pedestrian(data) 
#pedestrian_names = list(dict_pedestrian.keys())
#print(len(pedestrian_names))

In [17]:
#get_class_images("/home/sergio/Documentos/COCO_Dataset/train2017","/home/sergio/Documentos/COCO_Dataset/pedestrian_dataset_train",real_names)

# Filtrado de imágenes de validación

In [1]:
dataDir='/home/sergio/Documentos/COCO_Dataset'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
print(annFile)

/home/sergio/Documentos/COCO_Dataset/annotations/instances_val2017.json


In [3]:
from pycocotools.coco import COCO
import numpy as np
#import skimage.io as io
import matplotlib.pyplot as plt
#import pylab

# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=0.32s)
creating index...
index created!


In [7]:
catIds = coco.getCatIds(catNms=['person']);
imgIds = coco.getImgIds(catIds=catIds );

In [12]:
aux = coco.loadImgs(imgIds)
real_names =[]
for i in aux:
    real_names.append(i["file_name"])
print(len(real_names))
print(real_names[:12])

2693
['000000532481.jpg', '000000458755.jpg', '000000385029.jpg', '000000311303.jpg', '000000393226.jpg', '000000532493.jpg', '000000008211.jpg', '000000327701.jpg', '000000188439.jpg', '000000434204.jpg', '000000401446.jpg', '000000213033.jpg']


In [13]:
import os
os.chdir("/home/sergio/Documentos/COCO_Dataset")
os.mkdir("pedestrian_dataset_val")

In [14]:
#get_class_images("/home/sergio/Documentos/COCO_Dataset/val2017","/home/sergio/Documentos/COCO_Dataset/pedestrian_dataset_val",real_names)

## Creating Metadata train CSV

In [50]:
catIds = coco.getCatIds(catNms=['person']); #ID de la categoría persona
imgIds = coco.getImgIds(catIds=catIds ); # IDS de las imagenes con la categoria persona
annIds = coco.getAnnIds(imgIds=imgIds, catIds=catIds) #Anotaciones de las imágenes con la categoría persona
img_metadata = coco.loadImgs(imgIds) # Metadatos de las images de la categoría persona
img_annotations = coco.loadAnns(annIds) # Anotaciones de las instancias de la base de datos

In [51]:
img_metadata[0]

{'license': 2,
 'file_name': '000000262145.jpg',
 'coco_url': 'http://images.cocodataset.org/train2017/000000262145.jpg',
 'height': 427,
 'width': 640,
 'date_captured': '2013-11-20 02:07:55',
 'flickr_url': 'http://farm8.staticflickr.com/7187/6967031859_5f08387bde_z.jpg',
 'id': 262145}

In [65]:
id_to_file_name={}

for i in img_metadata:
    id_ = str(i["id"])
    file_name = i["file_name"]
    id_to_file_name[id_] = file_name

In [75]:
len(id_to_file_name)

64115

In [67]:
img_metadata_dict = {}

for i in img_metadata:
    img_metadata_dict[i["file_name"]] = {"width":i["width"],"height":i["height"]}

In [68]:
metadata_df = pd.DataFrame(img_metadata_dict).T

In [69]:
metadata_df.head()

,width,height
000000262145.jpg,640,427
000000262146.jpg,480,640
000000524291.jpg,640,426
000000262148.jpg,640,512
000000393223.jpg,640,480


In [70]:
os.chdir("/home/sergio/Documentos/COCO_Dataset")
metadata_df.to_csv("COCO_metadata_train.csv",index=True)

In [71]:
len(img_metadata_dict)

64115

## Creating Annotations COCO train CSV

In [72]:
len(img_annotations)

262465

In [76]:
dict_annotations = {}
num_instance = 0

#VER COMO SE GUARDARON LAS COORDENADAS DEL VOC DATASET, QUE CORDENADA (X,Y) SON

for index,j in enumerate(img_annotations):
    #print(j)
    dict_annotations[index] = {"file_name":id_to_file_name[str(j["image_id"])],
                                      "x":j["bbox"][0],
                                      "y":j["bbox"][1],
                                      "delta_x":j["bbox"][2],
                                     "delta_y": j["bbox"][3]}

In [78]:
annotations_df = pd.DataFrame(dict_annotations).T

In [79]:
annotations_df.head()

,file_name,x,y,delta_x,delta_y
0,000000262145.jpg,387,71.1,145,322
1,000000262145.jpg,494.31,91.04,11.7,16.16
2,000000262145.jpg,282.37,86.79,32.16,97.02
3,000000262145.jpg,526.17,44.14,90.5,188
4,000000262145.jpg,491.51,81.19,40.46,85.34


In [80]:
os.chdir("/home/sergio/Documentos/COCO_Dataset")
metadata_df.to_csv("COCO_annotations_train.csv",index=True)

## Creating metadata validation COCO

In [81]:
dataDir='/home/sergio/Documentos/COCO_Dataset'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
print(annFile)
coco  =COCO(annFile)

/home/sergio/Documentos/COCO_Dataset/annotations/instances_val2017.json
loading annotations into memory...
Done (t=0.27s)
creating index...
index created!


In [82]:
catIds = coco.getCatIds(catNms=['person']); #ID de la categoría persona
imgIds = coco.getImgIds(catIds=catIds ); # IDS de las imagenes con la categoria persona
annIds = coco.getAnnIds(imgIds=imgIds, catIds=catIds) #Anotaciones de las imágenes con la categoría persona
img_metadata = coco.loadImgs(imgIds) # Metadatos de las images de la categoría persona
img_annotations = coco.loadAnns(annIds) # Anotaciones de las instancias de la base de datos

In [83]:
id_to_file_name={}

for i in img_metadata:
    id_ = i["id"]
    file_name = i["file_name"]
    id_to_file_name[id_] = file_name

In [84]:
img_metadata_dict = {}

for i in img_metadata:
    img_metadata_dict[i["file_name"]] = {"width":i["width"],"height":i["height"]}

In [85]:
metadata_df = pd.DataFrame(img_metadata_dict).T

In [86]:
metadata_df.head()

,width,height
000000532481.jpg,640,426
000000458755.jpg,640,480
000000385029.jpg,640,480
000000311303.jpg,640,427
000000393226.jpg,640,480


In [87]:
os.chdir("/home/sergio/Documentos/COCO_Dataset")
metadata_df.to_csv("COCO_metadata_val.csv",index=True)

## Creating annotatios val COCO

In [89]:
id_to_file_name={}

for i in img_metadata:
    id_ = str(i["id"])
    file_name = i["file_name"]
    id_to_file_name[id_] = file_name

In [90]:
dict_annotations = {}
num_instance = 0

#VER COMO SE GUARDARON LAS COORDENADAS DEL VOC DATASET, QUE CORDENADA (X,Y) SON

for index,j in enumerate(img_annotations):
    #print(j)
    dict_annotations[index] = {"file_name":id_to_file_name[str(j["image_id"])],
                                      "x":j["bbox"][0],
                                      "y":j["bbox"][1],
                                      "delta_x":j["bbox"][2],
                                     "delta_y": j["bbox"][3]}

In [92]:
len(dict_annotations)

11004

In [93]:
annotations_df = pd.DataFrame(dict_annotations).T

In [94]:
annotations_df.head()

,file_name,x,y,delta_x,delta_y
0,000000532481.jpg,250.82,168.26,70.11,64.88
1,000000532481.jpg,285.55,370.56,12.07,19.21
2,000000458755.jpg,69.03,37.75,508.05,435.78
3,000000458755.jpg,567,1.27,73,86.86
4,000000458755.jpg,590.02,91.69,49.62,97.07


In [95]:
os.chdir("/home/sergio/Documentos/COCO_Dataset")
metadata_df.to_csv("COCO_annotations_val.csv",index=True)